In [ ]:
# ---------------------------------------------------------
# # ============================
# Name:Delight Mapololo
# Reg No. : R229476R
# Date: Nov 27, 2025
# Project: House Price Prediction using 21 CNN models
# ============================
# ---------------------------------------------------------


In [1]:

import os
import time
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Active computation device: {device}')
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    torch.cuda.manual_seed_all(42)

torch.manual_seed(42)
np.random.seed(42)


Active computation device: cpu


In [3]:

# -----------------------------
# STEP 2: Load CSV + Image Folder
# -----------------------------
# Adjust this path to your CSV file
csv_path = r"C:\Users\Brendon\Desktop\Deep_Learning\ASSIGNMENTS\final_zimbabwe_property_listings_complete.csv"
print(f"Reading dataset from: {csv_path}")
property_data = pd.read_csv(csv_path)
print(f"Imported rows: {len(property_data)}")

# The image folder you originally used in your code:
image_folder = Path(r"C:\Users\Brendon\Desktop\Deep_Learning\ASSIGNMENTS\images")
print(f"Image folder set to: {image_folder}")

# Attempt to detect image filename column in CSV
possible_image_cols = ['image_filenames', 'image_filename', 'image_path', 'filename', 'image', 'images']
image_col = None
for col in property_data.columns:
    if col.lower() in [c.lower() for c in possible_image_cols]:
        image_col = col
        break
if image_col is None:
    # fallback: any column containing 'image' or 'photo'
    for col in property_data.columns:
        if 'image' in col.lower() or 'photo' in col.lower():
            image_col = col
            break
if image_col is None:
    raise KeyError("No image column detected in the CSV. Expected columns like image_filenames or image_path.")
print(f"Using CSV image column: {image_col}")

# Helper to resolve image path (your images are in the folder so prefer folder join)
def resolve_image_path_from_folder(cell_value):
    """
    If cell_value already contains a full path, prefer it (but we'll prefer images in image_folder).
    If a filename is provided, join with image_folder and check existence.
    If multiple filenames are present, take the first (split on '|' or ';').
    """
    if pd.isna(cell_value):
        return None

    if isinstance(cell_value, str):
        # pick first filename if multiple
        if '|' in cell_value:
            candidate = cell_value.split('|')[0].strip()
        elif ';' in cell_value:
            candidate = cell_value.split(';')[0].strip()
        else:
            candidate = cell_value.strip()
    else:
        candidate = str(cell_value)

    # If candidate looks like an absolute path and exists, return it
    cand_path = Path(candidate)
    if cand_path.exists():
        return cand_path

    # Otherwise, try joining with provided image_folder
    joined = image_folder / candidate
    if joined.exists():
        return joined

    # Try common extensions appended to candidate while joined to folder
    for ext in ['.jpg', '.jpeg', '.png', '.webp', '.bmp']:
        p = image_folder / (candidate + ext)
        if p.exists():
            return p

    # Not found
    return None

# Create 'image_location' column with resolved Path or None
property_data['image_location'] = property_data[image_col].apply(resolve_image_path_from_folder)
valid_images = property_data['image_location'].notna().sum()
print(f"Resolved image paths: {valid_images} / {len(property_data)} rows")

# Detect price column
price_col = None
for col in property_data.columns:
    if 'price' in col.lower() or 'value' in col.lower() or 'listing' in col.lower():
        price_col = col
        break
if price_col is None:
    raise KeyError("No price-like column found in CSV.")
print(f"Detected price column: {price_col}")

# Convert price to numeric and drop rows without price
property_data[price_col] = pd.to_numeric(property_data[price_col], errors='coerce')
property_data = property_data[~property_data[price_col].isna()].reset_index(drop=True)
print(f"Rows after dropping NaN prices: {len(property_data)}")

# Fit price normalizer (StandardScaler)
price_normalizer = StandardScaler()
price_normalizer.fit(property_data[[price_col]].values)
print("Price normalizer (StandardScaler) initialized.")


Reading dataset from: C:\Users\Brendon\Desktop\Deep_Learning\ASSIGNMENTS\final_zimbabwe_property_listings_complete.csv
Imported rows: 1613
Image folder set to: C:\Users\Brendon\Desktop\Deep_Learning\ASSIGNMENTS\images
Using CSV image column: image_filenames
Resolved image paths: 468 / 1613 rows
Detected price column: price
Rows after dropping NaN prices: 1610
Price normalizer (StandardScaler) initialized.


In [4]:

# -----------------------------
# STEP 3: Dataset Class
# -----------------------------
class PropertyValueDataset(Dataset):
    def __init__(self, dataframe, image_column, image_folder_path, preprocessing=None,
                 incorporate_features=True, value_normalizer=None):
        self.df = dataframe.reset_index(drop=True).copy()
        self.image_column = image_column
        self.image_folder = Path(image_folder_path)
        self.preprocessing = preprocessing
        self.incorporate_features = incorporate_features
        self.value_normalizer = value_normalizer

        # Numerical attributes detection (flexible)
        potential_numerical_cols = [
            'building_area(m²)', 'land_area(m²)', 'bedrooms', 'bathrooms', 'rooms',
            'area', 'size', 'building_area', 'land_area', 'square_feet', 'sqft',
            'bed', 'bath', 'room_count'
        ]
        self.numerical_attributes = [c for c in potential_numerical_cols if c in self.df.columns]
        if not self.numerical_attributes:
            self.df['dummy_feature'] = 1.0
            self.numerical_attributes = ['dummy_feature']
            print("⚠️ No numeric columns found -> added dummy_feature")

        # Geo encoding
        potential_location_cols = ['location', 'city', 'region', 'area', 'neighborhood', 'suburb', 'town']
        location_col = None
        for c in potential_location_cols:
            if c in self.df.columns:
                location_col = c
                break
        if location_col:
            self.geo_encoder = LabelEncoder()
            self.df['geo_encoded'] = self.geo_encoder.fit_transform(self.df[location_col].fillna('Unspecified').astype(str))
            print(f"Using location column '{location_col}' for geo encoding.")
        else:
            self.df['geo_encoded'] = 0
            print("No location column found -> using geo_encoded = 0")

        # Feature columns and scaler
        self.feature_columns = [c for c in self.numerical_attributes if c in self.df.columns] + ['geo_encoded']
        self.feature_scaler = StandardScaler()
        feature_matrix = self.df[self.feature_columns].fillna(0).astype(float).values
        self.feature_matrix = self.feature_scaler.fit_transform(feature_matrix)
        print(f"Feature matrix prepared with shape: {self.feature_matrix.shape}")

        # store price column name for target extraction (outer scope price_col used)
        self.price_column = price_col

        # Ensure image_location column exists and contains Path objects when resolvable
        if 'image_location' not in self.df.columns:
            self.df['image_location'] = None
        else:
            # keep Path objects or None
            def to_path(x):
                try:
                    if pd.isna(x):
                        return None
                    p = Path(x)
                    return p if p.exists() else None
                except Exception:
                    return None
            self.df['image_location'] = self.df['image_location'].apply(to_path)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Resolve image path (prefer precomputed image_location)
        img_path = None
        if row.get('image_location', None) is not None:
            img_path = row['image_location']
        else:
            raw = row[self.image_column]
            if isinstance(raw, str):
                cand = Path(raw)
                if cand.exists():
                    img_path = cand
                else:
                    joined = self.image_folder / raw
                    if joined.exists():
                        img_path = joined
                    else:
                        # try first part if cell contains multiple filenames
                        if '|' in raw:
                            candidate = raw.split('|')[0].strip()
                            joined2 = self.image_folder / candidate
                            if joined2.exists():
                                img_path = joined2
            else:
                img_path = None

        # Load image or fallback blank
        if img_path is not None:
            try:
                img = Image.open(img_path).convert('RGB')
            except Exception:
                img = Image.new('RGB', (224, 224), color='black')
        else:
            img = Image.new('RGB', (224, 224), color='black')

        if self.preprocessing is not None:
            img = self.preprocessing(img)

        # attribute features
        attributes = torch.tensor(self.feature_matrix[idx], dtype=torch.float32)

        # target (price), possibly normalized
        raw_price = float(row[self.price_column])
        if self.value_normalizer is not None:
            price_val = float(self.value_normalizer.transform([[raw_price]])[0][0])
        else:
            price_val = raw_price
        price_tensor = torch.tensor(price_val, dtype=torch.float32)

        if self.incorporate_features:
            return img, attributes, price_tensor
        else:
            return img, price_tensor


In [5]:

# -----------------------------
# STEP 4: Transforms and Dataloaders
# -----------------------------
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# Split data
train_df, temp_df = train_test_split(property_data, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
print(f"Train/Val/Test counts: {len(train_df)}, {len(val_df)}, {len(test_df)}")

# Create datasets (they will resolve images using the provided image_folder)
train_dataset = PropertyValueDataset(train_df, image_column=image_col, image_folder_path=image_folder,
                                    preprocessing=train_transforms, incorporate_features=True, value_normalizer=price_normalizer)
val_dataset = PropertyValueDataset(val_df, image_column=image_col, image_folder_path=image_folder,
                                   preprocessing=val_transforms, incorporate_features=True, value_normalizer=price_normalizer)
test_dataset = PropertyValueDataset(test_df, image_column=image_col, image_folder_path=image_folder,
                                    preprocessing=val_transforms, incorporate_features=True, value_normalizer=price_normalizer)

# DataLoader settings (Windows-friendly defaults)
BATCH_SIZE = 16
NUM_WORKERS = 0  # set to >0 on Linux for speed
PIN_MEMORY = torch.cuda.is_available()

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                         num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

print(f"DataLoaders created — train batches: {len(train_loader)}, val: {len(val_loader)}, test: {len(test_loader)}")


Train/Val/Test counts: 1127, 241, 242
Using location column 'location' for geo encoding.
Feature matrix prepared with shape: (1127, 5)
Using location column 'location' for geo encoding.
Feature matrix prepared with shape: (241, 5)
Using location column 'location' for geo encoding.
Feature matrix prepared with shape: (242, 5)
DataLoaders created — train batches: 71, val: 16, test: 16


In [6]:

# -----------------------------
# STEP 5: Model + Multimodal Network
# -----------------------------
class PropertyValuationNetwork(nn.Module):
    def __init__(self, base_architecture, feature_count=7, dropout_rate=0.5, architecture_variant='standard'):
        super().__init__()
        self.architecture_variant = architecture_variant
        self.base = base_architecture

        # Try to neutralize final classifier to extract embeddings
        # Different model families have different attribute names for final layers.
        # We'll handle common cases; fallback to identity and a sample forward to infer dim.
        try:
            if architecture_variant in ['vgg','alexnet']:
                # keep features & classifier trimmed
                self.base.classifier = nn.Sequential(*list(self.base.classifier.children())[:-1])
            elif architecture_variant == 'densenet':
                # densenet.classifier exists
                _ = None
            elif architecture_variant == 'inception':
                self.base.fc = nn.Identity()
                self.base.aux_logits = False
            else:
                # For resnet-like models, replace fc with identity
                if hasattr(self.base, 'fc'):
                    self.base.fc = nn.Identity()
        except Exception:
            pass

        # Infer visual_feature_dim with a sample forward
        with torch.no_grad():
            self.base.eval()
            sample_input = torch.zeros(1,3,224,224)
            try:
                sample_out = self.base(sample_input)
                visual_feature_dim = sample_out.view(1, -1).size(1)
            except Exception:
                # As a safe fallback
                visual_feature_dim = 2048

        # Multimodal head
        self.feature_integration = nn.Sequential(
            nn.Linear(visual_feature_dim + feature_count, 1536),
            nn.BatchNorm1d(1536),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.6),
            nn.Linear(1536, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.6),
            nn.Linear(1024, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.5),
            nn.Linear(768, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.4),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.2),
            nn.Linear(128, 1)
        )

        # weight init for linear layers
        for m in self.feature_integration.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)

    def forward(self, visual_input, attribute_features):
        # Get visual embedding
        if self.architecture_variant in ['vgg','alexnet']:
            visual_features = self.base(visual_input)
            visual_features = visual_features.view(visual_features.size(0), -1)
        elif self.architecture_variant == 'densenet':
            # densenet: use features->relu->adaptive_avg_pool2d->flatten
            feats = self.base.features(visual_input)
            feats = nn.functional.relu(feats, inplace=True)
            feats = nn.functional.adaptive_avg_pool2d(feats, (1,1))
            visual_features = torch.flatten(feats, 1)
        else:
            out = self.base(visual_input)
            visual_features = out.view(out.size(0), -1)

        fused = torch.cat([visual_features, attribute_features], dim=1)
        out = self.feature_integration(fused)
        return out.squeeze(1)

# Factory to initialize backbones and wrap into PropertyValuationNetwork
def initialize_neural_network(model_identifier, feature_dimensions=7):
    mapping = {
        'EfficientNet': (lambda: models.efficientnet_b0(pretrained=True), 'efficientnet'),
        'MobileNet-v2': (lambda: models.mobilenet_v2(pretrained=True), 'mobilenet'),
        'ResNet': (lambda: models.resnet50(pretrained=True), 'standard'),
        'DenseNet': (lambda: models.densenet121(pretrained=True), 'densenet'),
        'Xception': (lambda: models.resnet50(pretrained=True), 'standard'),  # substitute (torchvision lacks xception)
        'Inception-V3': (lambda: models.inception_v3(pretrained=True, aux_logits=True), 'inception'),
        'GoogleNet': (lambda: models.googlenet(pretrained=True, aux_logits=True), 'googlenet'),
        'VGG': (lambda: models.vgg16(pretrained=True), 'vgg'),
        'AlexNet': (lambda: models.alexnet(pretrained=True), 'alexnet'),
        'NIN': (lambda: models.vgg16(pretrained=True), 'vgg'),
        'ZFNet': (lambda: models.alexnet(pretrained=True), 'alexnet'),
        # many architectures not directly in torchvision are substituted with resnet50 as a backbone
        'Squeeze-and-Excitation': (lambda: models.resnet50(pretrained=True), 'standard'),
        'Residual-Attention': (lambda: models.resnet50(pretrained=True), 'standard'),
        'WideResNet': (lambda: models.resnet50(pretrained=True), 'standard'),
        'Inception-ResNet-v2': (lambda: models.inception_v3(pretrained=True, aux_logits=True), 'inception'),
        'Inception-V4': (lambda: models.inception_v3(pretrained=True, aux_logits=True), 'inception'),
        'Competitive-SE': (lambda: models.resnet50(pretrained=True), 'standard'),
        'HRNetV2': (lambda: models.resnet50(pretrained=True), 'standard'),
        'FractalNet': (lambda: models.resnet50(pretrained=True), 'standard'),
        'Highway': (lambda: models.resnet50(pretrained=True), 'standard'),
        'CapsuleNet': (lambda: models.resnet50(pretrained=True), 'standard'),
    }

    if model_identifier not in mapping:
        raise ValueError(f"Unknown architecture: {model_identifier}")

    backbone_init, arch_variant = mapping[model_identifier]
    backbone = backbone_init()
    net = PropertyValuationNetwork(backbone, feature_count=feature_dimensions, architecture_variant=arch_variant)
    return net.to(device)


In [7]:

# -----------------------------
# STEP 6: Training Function
# -----------------------------
def execute_model_training(network_name, training_loader, validation_loader, training_cycles=7, learning_rate=0.003):
    print(f"\n{'='*60}\nStarting training for: {network_name}\n{'='*60}")
    # determine feature_dimensions from loaders (datasets)
    # Expect dataset returns (img, attributes, price)
    if 'feature_dimensions' not in globals():
        try:
            sample = next(iter(training_loader))
            # sample[1] should be attribute tensor
            feature_dims = sample[1].shape[1]
        except Exception:
            # fallback to dataset.feature_matrix shape
            feature_dims = getattr(training_loader.dataset, 'feature_matrix').shape[1]
        globals()['feature_dimensions'] = feature_dims
        print(f"Feature dimensions set to: {feature_dims}")
    else:
        feature_dims = globals()['feature_dimensions']
        print(f"Using existing feature_dimensions: {feature_dims}")

    # Build model
    nn_start = time.time()
    try:
        model = initialize_neural_network(network_name, feature_dimensions=feature_dims)
    except Exception as e:
        print(f"Failed to initialize {network_name}: {e}")
        raise

    print("Model initialized. Params:", sum(p.numel() for p in model.parameters()))
    # Loss and optimizer
    l1 = nn.L1Loss()
    mse = nn.MSELoss()
    def composite_loss(pred, tgt):
        return 0.5 * l1(pred, tgt) + 0.5 * mse(pred, tgt)

    optimizer = optim.AdamW(model.parameters(), lr=learning_rate * 0.7, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=6, min_lr=1e-6, cooldown=2)
    grad_scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None

    # adjust cycles for heavy nets
    heavy = ['ResNet', 'VGG', 'Inception-V3', 'Inception-ResNet-v2', 'Inception-V4', 'DenseNet']
    if network_name in heavy:
        training_cycles = min(training_cycles, 7)
        print(f"Resource-intensive architecture -> cycles set to {training_cycles}")

    history = {'training_loss': [], 'validation_loss': []}
    best_val_loss = float('inf')
    best_state = None
    patience = 10
    wait = 0
    t0 = time.time()

    for epoch in range(training_cycles):
        model.train()
        train_loss = 0.0
        n_batches = 0
        loop = tqdm(training_loader, desc=f"{network_name} Epoch {epoch+1}/{training_cycles}", ncols=100)
        for imgs, attrs, targets in loop:
            imgs = imgs.to(device, non_blocking=True)
            attrs = attrs.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)

            optimizer.zero_grad()
            if grad_scaler:
                with torch.cuda.amp.autocast():
                    preds = model(imgs, attrs)
                    loss = composite_loss(preds, targets)
                grad_scaler.scale(loss).backward()
                grad_scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                grad_scaler.step(optimizer)
                grad_scaler.update()
            else:
                preds = model(imgs, attrs)
                loss = composite_loss(preds, targets)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

            train_loss += loss.item()
            n_batches += 1
            if n_batches % 10 == 0:
                loop.set_postfix({'batch_loss': f"{loss.item():.4f}", 'avg_loss': f"{train_loss/n_batches:.4f}"})

        avg_train_loss = train_loss / max(1, n_batches)

        # Validation
        model.eval()
        val_loss = 0.0
        preds_collect = []
        truth_collect = []
        with torch.no_grad():
            for imgs, attrs, targets in validation_loader:
                imgs = imgs.to(device, non_blocking=True)
                attrs = attrs.to(device, non_blocking=True)
                targets = targets.to(device, non_blocking=True)

                if grad_scaler:
                    with torch.cuda.amp.autocast():
                        preds = model(imgs, attrs)
                else:
                    preds = model(imgs, attrs)

                loss = composite_loss(preds, targets)
                val_loss += loss.item()
                preds_collect.extend(preds.cpu().numpy())
                truth_collect.extend(targets.cpu().numpy())

        avg_val_loss = val_loss / max(1, len(validation_loader))
        # denormalize for metrics if needed
        preds_arr = np.array(preds_collect).reshape(-1,1)
        truth_arr = np.array(truth_collect).reshape(-1,1)
        if hasattr(validation_loader.dataset, 'value_normalizer') and validation_loader.dataset.value_normalizer is not None:
            den_preds = validation_loader.dataset.value_normalizer.inverse_transform(preds_arr).flatten()
            den_truth = validation_loader.dataset.value_normalizer.inverse_transform(truth_arr).flatten()
        else:
            den_preds = preds_arr.flatten()
            den_truth = truth_arr.flatten()

        val_r2 = r2_score(den_truth, den_preds) if len(den_preds)>0 else float('nan')
        val_rmse = np.sqrt(mean_squared_error(den_truth, den_preds)) if len(den_preds)>0 else float('nan')

        scheduler.step(avg_val_loss)
        history['training_loss'].append(avg_train_loss)
        history['validation_loss'].append(avg_val_loss)

        print(f"Epoch {epoch+1}/{training_cycles} | TrainLoss={avg_train_loss:.6f} | ValLoss={avg_val_loss:.6f} | ValR2={val_r2:.4f} | ValRMSE={val_rmse:.2f}")

        # early stopping bookkeeping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    total_train_time = time.time() - t0

    # restore best weights
    if best_state is not None:
        model.load_state_dict(best_state)

    # Final evaluation on test_loader
    model.eval()
    test_preds = []
    test_truths = []
    with torch.no_grad():
        for imgs, attrs, targets in test_loader:
            imgs = imgs.to(device, non_blocking=True)
            attrs = attrs.to(device, non_blocking=True)
            preds = model(imgs, attrs)
            test_preds.extend(preds.cpu().numpy())
            test_truths.extend(targets.cpu().numpy())

    test_preds = np.array(test_preds).reshape(-1,1)
    test_truths = np.array(test_truths).reshape(-1,1)
    if hasattr(test_loader.dataset, 'value_normalizer') and test_loader.dataset.value_normalizer is not None:
        final_preds = test_loader.dataset.value_normalizer.inverse_transform(test_preds).flatten()
        final_truths = test_loader.dataset.value_normalizer.inverse_transform(test_truths).flatten()
    else:
        final_preds = test_preds.flatten()
        final_truths = test_truths.flatten()

    eval_r2 = r2_score(final_truths, final_preds) if len(final_preds)>0 else float('nan')
    eval_rmse = np.sqrt(mean_squared_error(final_truths, final_preds)) if len(final_preds)>0 else float('nan')
    eval_mse = mean_squared_error(final_truths, final_preds) if len(final_preds)>0 else float('nan')
    eval_mae = mean_absolute_error(final_truths, final_preds) if len(final_preds)>0 else float('nan')

    metrics = {
        'architecture_name': network_name,
        'evaluation_r2': eval_r2,
        'evaluation_rmse': eval_rmse,
        'evaluation_mse': eval_mse,
        'evaluation_mae': eval_mae,
        'optimal_validation_loss': best_val_loss,
        'training_duration': total_train_time,
        'training_history': history,
        'model_parameters': best_state
    }

    print(f"\n{network_name} summary: R2={eval_r2:.4f}, RMSE={eval_rmse:.2f}, Training time={total_train_time:.2f}s")
    return model, metrics


In [ ]:

# -----------------------------
# STEP 7: Train All Models
# -----------------------------
model_names = [
    'EfficientNet', 'MobileNet-v2', 'ResNet', 'DenseNet', 'Xception',
    'Inception-V3', 'GoogleNet', 'VGG', 'Squeeze-and-Excitation',
    'Residual-Attention', 'WideResNet', 'Inception-ResNet-v2',
    'Inception-V4', 'Competitive-SE', 'HRNetV2', 'FractalNet',
    'Highway', 'AlexNet', 'NIN', 'ZFNet', 'CapsuleNet'
]

trained_models = {}
complete_performance_metrics = []

# Auto-detect train/val/test loader variables already created above
# (Our variables are train_loader, val_loader, test_loader)
if 'feature_dimensions' not in globals():
    try:
        sample_batch = next(iter(train_loader))
        globals()['feature_dimensions'] = sample_batch[1].shape[1]
        print(f"Set feature_dimensions = {globals()['feature_dimensions']}")
    except Exception:
        # fallback to dataset feature matrix
        globals()['feature_dimensions'] = getattr(train_loader.dataset, 'feature_matrix').shape[1]
        print(f"Fallback feature_dimensions = {globals()['feature_dimensions']}")
else:
    print(f"Using existing feature_dimensions = {globals()['feature_dimensions']}")

print("\n" + "="*80)
print(f"🚀 Starting training of {len(model_names)} models on {device}")
print("="*80)

for i, name in enumerate(model_names, start=1):
    print(f"\n[{i}/{len(model_names)}] -> {name}")
    start = time.time()
    try:
        model, metrics = execute_model_training(
            network_name=name,
            training_loader=train_loader,
            validation_loader=val_loader,
            training_cycles=7,
            learning_rate=0.003
        )
        elapsed = time.time() - start
        trained_models[name] = model
        complete_performance_metrics.append(metrics)

        # Save the final trained model state + metrics
        safe_name = name.replace(" ", "").replace("-", "")
        file_name = f"best_{safe_name}.pth"
        torch.save({
            'model_state_dict': model.state_dict(),
            'architecture_name': name,
            'metrics': metrics,
            'feature_dimensions': globals().get('feature_dimensions', None)
        }, file_name)
        print(f"Saved {file_name} (elapsed {elapsed/60:.2f} min)")

    except Exception as e:
        print(f"Error training {name}: {e}")
        continue

print("\nAll training loops finished.")
print(f"Successfully trained {len(complete_performance_metrics)} models.")


Set feature_dimensions = 5

🚀 Starting training of 21 models on cpu

[1/21] -> EfficientNet

Starting training for: EfficientNet
Using existing feature_dimensions: 5
Model initialized. Params: 9761381


EfficientNet Epoch 1/7: 100%|███| 71/71 [04:24<00:00,  3.72s/it, batch_loss=0.0927, avg_loss=0.3387]


Epoch 1/7 | TrainLoss=0.337048 | ValLoss=3.682741 | ValR2=-0.1150 | ValRMSE=2249356373147.15


EfficientNet Epoch 2/7: 100%|███| 71/71 [06:00<00:00,  5.07s/it, batch_loss=0.0246, avg_loss=0.1128]


Epoch 2/7 | TrainLoss=0.114801 | ValLoss=3.419084 | ValR2=-0.0482 | ValRMSE=2180941589773.40


...

100%|██████████| 30.8M/30.8M [00:15<00:00, 2.07MB/s]


Model initialized. Params: 12451689
Resource-intensive architecture -> cycles set to 7


DenseNet Epoch 1/7:   0%|                                                    | 0/71 [00:02<?, ?it/s]


Error training DenseNet: mat1 and mat2 shapes cannot be multiplied (16x1029 and 1005x1536)

[5/21] -> Xception

Starting training for: Xception
Using existing feature_dimensions: 5
Model initialized. Params: 29590593


Xception Epoch 1/7:  27%|█▊     | 19/71 [02:04<05:12,  6.01s/it, batch_loss=0.2960, avg_loss=0.5972]

In [ ]:

# -----------------------------
# STEP 8: Evaluation & Comparison
# -----------------------------
if len(complete_performance_metrics) == 0:
    print("No successful trainings recorded.")
    performance_comparison_df = pd.DataFrame(columns=['Architecture','R² Score','RMSE','MSE','MAE','Training Duration (s)'])
else:
    performance_comparison_df = pd.DataFrame([
        {
            'Architecture': m['architecture_name'],
            'R² Score': m['evaluation_r2'],
            'RMSE': m['evaluation_rmse'],
            'MSE': m['evaluation_mse'],
            'MAE': m['evaluation_mae'],
            'Training Duration (s)': m['training_duration']
        } for m in complete_performance_metrics
    ])
    if 'R² Score' in performance_comparison_df.columns and len(performance_comparison_df)>0:
        performance_comparison_df = performance_comparison_df.sort_values('R² Score', ascending=False).reset_index(drop=True)

print("\n" + "="*80)
print("NEURAL NETWORK ARCHITECTURE PERFORMANCE COMPARISON")
print("="*80)
if len(performance_comparison_df) > 0:
    print(performance_comparison_df.to_string(index=False))
else:
    print("No performance data to show.")

# Determine best architecture (if any)
if len(performance_comparison_df) > 0:
    best_arch = performance_comparison_df.iloc[0]['Architecture']
    best_metrics = next((m for m in complete_performance_metrics if m['architecture_name']==best_arch), None)
    print(f"\n🏆 Best architecture: {best_arch}")
    if best_metrics:
        print(f"  R²: {best_metrics['evaluation_r2']:.4f}, RMSE: {best_metrics['evaluation_rmse']:.2f}, MAE: {best_metrics['evaluation_mae']:.2f}")
else:
    best_arch = None
    best_metrics = None
    print("No best architecture determined.")

# Save comparison CSV
performance_comparison_df.to_csv('architecture_performance_comparison.csv', index=False)
print("Saved architecture_performance_comparison.csv")

# Save best model artifact
if best_arch and best_arch in trained_models:
    save_obj = {
        'architecture': best_arch,
        'metrics': best_metrics,
        'model_state_dict': trained_models[best_arch].state_dict(),
        'complete_metrics': complete_performance_metrics
    }
    torch.save(save_obj, 'optimal_property_valuation_network.pth')
    print("Saved optimal_property_valuation_network.pth")


In [ ]:

# -----------------------------
# STEP 9: Visualization
# -----------------------------
if len(performance_comparison_df) > 0:
    fig, axes = plt.subplots(2,2, figsize=(16,12))
    ax1 = axes[0,0]
    ax1.barh(performance_comparison_df['Architecture'], performance_comparison_df['R² Score'])
    ax1.set_title('R² Score by Architecture')
    ax1.set_xlabel('R² Score')

    ax2 = axes[0,1]
    ax2.barh(performance_comparison_df['Architecture'], performance_comparison_df['RMSE'])
    ax2.set_title('RMSE by Architecture')
    ax2.set_xlabel('RMSE (USD)')

    ax3 = axes[1,0]
    ax3.barh(performance_comparison_df['Architecture'], performance_comparison_df['Training Duration (s)'])
    ax3.set_title('Training Duration (s)')
    ax3.set_xlabel('Seconds')

    ax4 = axes[1,1]
    scatter = ax4.scatter(performance_comparison_df['RMSE'], performance_comparison_df['R² Score'],
                          s=120, c=performance_comparison_df['Training Duration (s)'], cmap='plasma')
    ax4.set_xlabel('RMSE')
    ax4.set_ylabel('R² Score')
    plt.colorbar(scatter, ax=ax4, label='Training Duration (s)')

    plt.tight_layout()
    out_img = 'architecture_performance_comparison_visualization.png'
    plt.savefig(out_img, dpi=300, bbox_inches='tight')
    print(f"Saved {out_img}")
    plt.show()

print("\n🎉 Pipeline complete. Artifacts produced:")
print(" - architecture_performance_comparison.csv")
print(" - architecture_performance_comparison_visualization.png")
print(" - best_<architecture>.pth files for successfully trained models")
print(" - optimal_property_valuation_network.pth (if a best model exists)")

